# DATA CLEANING

# TRI Clinical Trial
###### <font color='gray'>By: Jeonwook Kang

---

In [ ]:
import pandas as pd
import sqlite3 as sql
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# import csv file into pandas dataframe

adverse_data = pd.read_csv("data/adae.csv")
other_data = pd.read_csv("data/ADLBC.csv")

## Data Cleaning/Exploratory Data Analysis

In [ ]:
# checking for the number of unique clinical trial subjects with adverse effects

adverse_count = list(adverse_data["usubjid"].drop_duplicates())
len(adverse_count)

In [ ]:
# checking for the total number of unique clinical trial subjects

total_count = list(other_data["usubjid"].drop_duplicates())
len(total_count)

In [ ]:
# confirming that all the subjects in adae dataset is also included in the overall data

checking = total_count.copy()
for x in adverse_count:
    if x in total_count:
        checking.remove(x)
len(checking)

In [ ]:
(225/254) * 100

**Note:**
Out of 254 subjects, 225 subjects (88.6%) is reported to have some type of adverse effect.

In [ ]:
# checking for number of unique items in each column to determine which columns are to be considered

dicts = {}

for x in adverse_data.columns:
    dicts[x] = len(adverse_data[x].value_counts())
dicts

**Note:** Above information represents unique item in each column. Columns with too many unique items will be excluded for analysis purposes. For example, *aedecod*(specific conditions) has 242 unique conditions so *aebodsys*(Body System) which has only 23 unique systems will be used instead. 

In [ ]:
dict(adverse_data['llterm'].value_counts())

In [ ]:
# checking for any missing values

adverse_data.isna().sum()

**Note:**
*aesev*(Severity/Intensity) and *aerel*(Causality) is the top two indicator of an adverse effect related to the use of Xanomeline. Other relavant columns include *trtp*(Control/Experimental Group) and *agegrp*(Age group). As we can see from above, only 4 rows are missing in the causality column, thus those missing rows will be excluded from the analysis.

In [ ]:
# filtering out and cleaning the dataframe with target columns

adverse_data = adverse_data[["subjid", "aeser", "aesev", "aerel", "trtp", "aebodsys", "agegrp"]]
adverse_data.dropna(inplace=True)
adverse_data.head(20)

**Note:** After filtering out the target columns we can see that there are multiple records of adverse effect from a single subject which could cause a weighted bias in the analysis. Therefore the dataframe will be further engineered to contain only the highest priority row for each subject. The priority was determined by using *aesev* (Intensity) column.

In [ ]:
# selecting the row with highest priority and dropping rows with other duplicate subjects
# higher order of selection is made by aerel(causality) column

subject = list(adverse_data["subjid"].drop_duplicates())

df = pd.DataFrame()
for x in subject:
    df_initial = adverse_data[adverse_data["subjid"] == x]
    if len(df_initial) > 1:
        if "SEVERE" in list(df_initial["aesev"]):
            df_temp = df_initial[df_initial["aesev"] == "SEVERE"]
            df_temp = df_temp.sort_values("aerel")
            df_temp = df_temp.drop_duplicates(subset = ['subjid'], keep = "last")
        elif "MODERATE" in list(df_initial["aesev"]):
            df_temp = df_initial[(df_initial["aesev"] == "MODERATE")]
            df_temp = df_temp.sort_values("aerel")
            df_temp = df_temp.drop_duplicates(subset = ["subjid"], keep = "last")
        else:
            df_temp = df_initial.sort_values("aerel")
            df_temp = df_initial.drop_duplicates(subset = ["subjid"], keep = "last")
        df = df.append(df_temp)
    elif len(df_initial) == 1:
        df = df.append(df_initial)
        
df.reset_index(inplace=True)
df.drop(columns=["index"], inplace=True)
df

In [ ]:
df["aesev"].value_counts()

In [ ]:
adverse_data.dropna(inplace=True)

In [ ]:
adverse_data.head()

In [ ]:
df["aerel"].value_counts()

In [ ]:
subject = list(adverse_data["subjid"].drop_duplicates())

In [ ]:
df["trtp"].value_counts()

**Note:** Since we are more interested in the possible causal effect between Xanomeline use and it's adversary effects, rows with None value in the aerel(causality) column are intentionally dropped if there were alternative values in that column. However, this was done only for the subject who experienced more than one effects.

In [ ]:
# exporting final cleaned dataset into the file

df.to_csv("data/cleaned_adae.csv")

In [ ]:
(79+76)/(79+76+69)

**Note:** Out of the subjects that experienced adversary effects, 69% of them were in the treatment group. It's evident that the remaining 31% of the subjects experienced a placebo effect.

---

## Statistical Analysis

In [ ]:
# assigned numerical values to severity and causality column to compare treatment and control group

df['severity_number'] = df.aesev.map({"MILD":0, "MODERATE":1, "SEVERE":2})
df['causality_number'] = df.aerel.map({"NONE":0, "REMOTE":1, "POSSIBLE":2, "PROBABLE":3})
df

In [ ]:
# filtered out necessary components from the complete dataframe

df_placebo = df[df['trtp'] == "Placebo"][['severity_number','causality_number']]
df_lowdose = df[df['trtp'] == "Xanomeline Low Dose"][['severity_number','causality_number']]
df_highdose = df[df['trtp'] == "Xanomeline High Dose"][['severity_number','causality_number']]

In [ ]:
# forming a array of numbers to perform two sample t-test to check for statistical significance

sn_placebo = np.array(df_placebo['severity_number'])
sn_lowdose = np.array(df_lowdose['severity_number'])
sn_highdose = np.array(df_highdose['severity_number'])

cn_placebo = np.array(df_placebo['causality_number'])
cn_lowdose = np.array(df_lowdose['causality_number'])
cn_highdose = np.array(df_highdose['causality_number'])

In [ ]:
# test between control and lowdose treatment group for severity measure

stats.ttest_ind(sn_placebo, sn_lowdose)

In [ ]:
# test between control and highdose treatment group for severity measure
stats.ttest_ind(sn_placebo, sn_highdose)

In [ ]:
# test between control and lowdose treatment group for causality measure

stats.ttest_ind(cn_placebo, cn_lowdose)

In [ ]:
# test between control and highdose treatment group for causality measures
stats.ttest_ind(cn_placebo, cn_highdose)

In [ ]:
def change_width(ax, new_value) :
    for patch in ax.patches :
        current_width = patch.get_width()
        diff = current_width - new_value
        patch.set_width(new_value)
        patch.set_x(patch.get_x() + diff * .5)

fig, (ax1, ax2) = plt.subplots(figsize=(13,6), ncols=2, dpi=200)

x= ["placebo", "low-dose", "high-dose"]
y= [sn_placebo.mean(), sn_lowdose.mean(), sn_highdose.mean()]
y1= [cn_placebo.mean(), cn_lowdose.mean(), cn_highdose.mean()]

sns.barplot(x=x, y=y, ax=ax1)
ax1.set_title('Mean of Severity Score', fontweight='bold', fontsize=15)
ax1.set_xlabel('Experimental Group', fontweight='bold', fontsize=12)
ax1.set_ylabel('Mean Value', fontweight='bold', fontsize=12)
change_width(ax1, 0.7)

sns.barplot(x=x, y=y1, ax=ax2)
ax2.set_title('Mean of Causality Score', fontweight='bold', fontsize=15)
ax2.set_xlabel('Experimental Group', fontweight='bold', fontsize=12)
ax2.set_ylabel('Mean Value', fontweight='bold', fontsize=12);
change_width(ax2, 0.7)

plt.subplots_adjust(wspace=.3)

fig.suptitle("Measure Scores Comparison", fontweight='bold', fontsize=20);

**Note:** In this statistical analysis section, we first assigned a numerical values to each items in the severity and causality column. The reason is that we can use this numerical values to check for statistical significance in the difference of the mean between control and treatment groups. As seen from the p-value, every single test came out to be significant with an extremely small p-value. Therefore it is not safe to continue the clinical trial since there is a clear relationship between the adverse events and the drug use.

---

## SQL Query

In [ ]:
conn = sql.Connection('data/data.sqlite')

In [ ]:
# df.to_sql('cleann', conn)

# Just need to pass this arguement once to create a sql table

In [ ]:
q = """
SELECT
    aesev AS 'Maximum Severity',
    COUNT(*) FILTER (WHERE agegrp LIKE '%<65%') AS '<65',
    COUNT(*) FILTER (WHERE agegrp LIKE '%65-80%') AS '65-80',
    COUNT(*) FILTER (WHERE agegrp LIKE '%>80%') AS '>80',
    COUNT(subjid) AS 'Total'
FROM cleann
GROUP BY aesev


"""

table = pd.read_sql(q,conn)
table = pd.DataFrame(table)
table

In [ ]:
df1 = {'Maximum Severity': 'ANY SEVERITY', '<65' : 29, '65-80': 127, '>80':68, 'Total': 224}
final = table.append(df1, ignore_index=True)
final.set_index('Maximum Severity', inplace=True)
final